# AugmentedGPT Playground

Please run `poetry install` under the project root directory. This will install all the necessary dependencies for you.

## Imports and set API key

In [1]:
from agentia import Agent
from typing import Annotated, Literal
import dotenv  # type: ignore

dotenv.load_dotenv()

True

## Create an agent with tools.

In [2]:
def get_weather(
    location: Annotated[str, "The city and state, e.g. San Francisco, CA"],
    unit: Annotated[
        Literal["celsius", "fahrenheit"], "The unit for the temperature"
    ] = "fahrenheit",
):
    """Get the current weather in a given location"""
    return {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }


agent = Agent(tools=[get_weather])

response = await agent.chat_completion("What is the weather like in boston?")

print(response)

The weather in Boston, MA is currently 72°F with sunny and windy conditions.
